In [1]:
# read saved results (tables) of old runs
# combine multiple runs
# make significance plots

import sys
import pandas as pd
import numpy as np
sys.path.append('../../') # to access the files in higher directories
sys.path.append('../') # to access the files in higher directories
import pandas as pd
import core_tools as ct
import core_calib as cal
from scipy import stats


In [2]:
main_path = './Data manipulation/results'
# exp_names = ["1692118038_T10", "1692121657_T50", "1692130088_T100", "1692161204_O50", "1692172225_O100"] # , "1692107939_Z8T0"
# exp_names = ["1692351821_RFseed"]
# exp_names = ["1698157809_Paper_v3_Real30", "1698169547_Paper_v3_Real30_Laplace"]
exp_names = ["1714153141_Real30_CTfix4", "1714161293_CT_L1"]

exp_comp = "_L"

In [3]:
# to combine Calib and OOB runs in one CD plot
tables_all = {}
for exp in exp_names:
    path = f"{main_path}/{exp}"
    dict_data, tables, params = ct.read_results(path)
    if len(tables_all) == 0:
        tables_all = tables
    else: # combine tables
        for table_key in tables.keys():
            col_names = np.array((tables[table_key].columns))
            col_names += exp_comp
            tables[table_key].columns = col_names
            # print(tables[table_key])
            # print(f"key {table_key} col_names", col_names)
            tables_all[table_key] = pd.concat([tables_all[table_key], tables[table_key]], axis=1)
            # print(tables_all[table_key])

# tables_all = ct.add_rank_mean(tables_all)
tables_all["brier"]

# ct.res_statistics(tables_all, params["metrics"], ".")

,RF_d,RF_opt,RF_large,Platt,ISO,Beta,VA,CT,PPA,Rank,RF_d_L,RF_opt_L,RF_large_L,Platt_L,ISO_L,Beta_L,VA_L,CT_L,PPA_L,Rank_L
Data,,,,,,,,,,,,,,,,,,,,
cm1,0.093479,0.087034,0.087991,0.087369,0.089137,8.614854e-02,0.090104,0.088035,8.772971e-02,0.092853,0.088542,0.087034,0.085832,0.087076,0.089003,8.589856e-02,0.089889,0.086788,0.087147,0.092853
datatrieve,0.086600,0.082124,0.081548,0.081790,0.088697,8.471090e-02,0.090891,0.080060,7.987189e-02,0.084713,0.079490,0.082224,0.077246,0.080350,0.088866,8.542679e-02,0.093051,0.077834,0.079351,0.085882
kc1_class_level_defectiveornot,0.188345,0.173202,0.175868,0.174725,0.183433,1.784802e-01,0.174653,0.187003,1.759579e-01,0.205872,0.180576,0.176751,0.172489,0.175792,0.184323,1.781477e-01,0.175855,0.181353,0.175753,0.208021
kc1,0.110310,0.108572,0.104137,0.110551,0.110906,1.096874e-01,0.110237,0.110355,1.096148e-01,0.113501,0.106778,0.108849,0.103358,0.109581,0.110537,1.085429e-01,0.109906,0.110494,0.108933,0.112538
kc2,0.128132,0.120310,0.121712,0.120101,0.122748,1.209866e-01,0.123538,0.120963,1.204024e-01,0.126999,0.123798,0.119969,0.120376,0.120570,0.127272,1.216281e-01,0.125228,0.118990,0.120206,0.128424
kc3,0.079230,0.076104,0.075165,0.077456,0.080615,7.678826e-02,0.080699,0.078970,7.951837e-02,0.078255,0.076192,0.075950,0.074054,0.076091,0.080159,7.607542e-02,0.078715,0.078051,0.077009,0.078114
pc1,0.054291,0.053685,0.050679,0.055613,0.057178,5.635256e-02,0.055955,0.056346,5.562198e-02,0.056416,0.052162,0.054130,0.050297,0.056052,0.058653,5.636315e-02,0.057203,0.055311,0.055476,0.058514
spect,0.137221,0.120148,0.128855,0.126234,0.130845,1.277475e-01,0.128761,0.124053,1.257971e-01,0.153752,0.130473,0.125306,0.125963,0.123583,0.133051,1.258519e-01,0.129848,0.124246,0.124924,0.148613
spectf,0.139699,0.131512,0.129171,0.135515,0.140768,1.350741e-01,0.137396,0.134934,1.364251e-01,0.157134,0.134853,0.133262,0.128077,0.137560,0.138986,1.361880e-01,0.135693,0.135743,0.137384,0.155514


In [4]:

def stat_test_binary(test_table, method, metric, log=False):

    # Create two sample arrays
    Normal = np.array(test_table.iloc[:, 0])
    Modified = np.array(test_table.iloc[:, 1])

    result = Normal - Modified # for loss comparison
    # result = Modified - Normal # for acc comparison
    if metric == "acc":
        result[result > 0] = 0
        result[result < 0] = 1
    else:
        result[result > 0] = 1
        result[result < 0] = 0
    n_trials = len(result) 
    observed_successes = int(sum(result))

    if log:
        print(f"{metric} {method} observed_successes", observed_successes)

    result = stats.binomtest(observed_successes, n=n_trials, p=0.5, alternative='less')
    p_value = result.pvalue

    if log:
        print("p_value", p_value)

    alpha = 0.05

    if p_value < alpha:
        if log:
            print("Reject the null hypothesis. The observed frequency is significantly different from the expected frequency.")
        return 1
    else:
        if log:
            print("Fail to reject the null hypothesis. There is no significant difference.")
        return 0

def stat_test_ttest(test_table, method, metric, log=False):

    if metric == "acc":
        statistic, p_value = stats.ttest_rel(test_table.iloc[:, 0], test_table.iloc[:, 1], alternative='less')
    else:
        statistic, p_value = stats.ttest_rel(test_table.iloc[:, 0], test_table.iloc[:, 1], alternative='greater')


    if log:
        print("p_value", p_value)
    alpha = 0.05

    if p_value < alpha:
        if log:
            # print("Reject the null hypothesis. The observed frequency is significantly different from the expected frequency.")
            print("The change creates significant improvment")
        return '\ding{51}'
    else:
        if metric == "acc":
            statistic, p_value = stats.ttest_rel(test_table.iloc[:, 0], test_table.iloc[:, 1], alternative='greater')
        else:
            statistic, p_value = stats.ttest_rel(test_table.iloc[:, 0], test_table.iloc[:, 1], alternative='less')

        if p_value < alpha:
            if log:
                print("The change creates significant decline.")
            return '\ding{55}'
        else:
            if log:    
                print("Fail to reject the null hypothesis. There is no significant difference.")
        return "  "



In [6]:
calib_methods = params["calib_methods"]

metrics =  ["acc", "brier", "logloss", "ece"]

res_table = []
for method in calib_methods:
    res_array = [method]
    for metric in metrics:
        test_table = tables_all[metric][[method, method + exp_comp]]

        # if method == "RF_opt" and metric == "brier":
        #     print(test_table)

        res_array.append(stat_test_ttest(test_table, method, metric))
    res_table.append(res_array)

res_table = np.array(res_table)

res_table = pd.DataFrame(res_table, columns=["calib_methods"]+ metrics)
res_table.set_index('calib_methods', inplace=True)
res_table.to_latex(f"./comp_stat_test.txt")
res_table

/var/folders/gm/nlbfzh25075c0jwn9khghrfw0000gn/T/ipykernel_5814/2042370032.py:21: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  res_table.to_latex(f"./comp_stat_test.txt")


,acc,brier,logloss,ece
calib_methods,,,,
RF_d,\ding{51},\ding{51},\ding{51},
RF_opt,,\ding{55},,
RF_large,,,\ding{51},\ding{51}
Platt,,,\ding{51},
ISO,\ding{55},,,
Beta,,\ding{51},\ding{51},
VA,\ding{55},,,
CT,,,\ding{51},\ding{51}
PPA,,,\ding{51},\ding{51}
